# Build human phosphoproteome dataset

## Setup

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
from katlas.core import *
import seaborn as sns
from tqdm import tqdm
import numpy as np

tqdm.pandas()

## Load PhosphoSitePlus and Ochoa et al.

In [3]:
ochoa = Data.get_ochoa_site()
psp = Data.get_psp_human_site()

Both sets' gene column contains nan

PSP is gene name, ochoa is porotein name

## PSP

In [4]:
psp = psp[psp.site.str[0].isin(['S','T','Y'])]

In [5]:
psp = psp[psp.LT_LIT.notna()].reset_index(drop=True)

In [6]:
psp['site_seq'] = psp['site_seq'].str.upper()

In [7]:
psp = psp[['gene','uniprot','site','site_seq']]

## Ochoa

In [8]:
ochoa['site'] = ochoa.residue+ochoa.position.astype(str)

In [9]:
ochoa = ochoa[['gene','current_uniprot','site','site_seq']]

In [10]:
ochoa = ochoa.rename(columns={'current_uniprot':'uniprot'})

## Combine

In [11]:
ochoa['source']='ochoa'

In [12]:
psp['source']='psp'

In [13]:
comb = psp.merge(ochoa,how='outer',on=['uniprot','site'],suffixes=('_psp','_ochoa'))

In [14]:
def join_columns(row, columns):
    "Join non-NA values from specified columns with a separator"
    valid_values = set(row[col] for col in columns if pd.notna(row[col]))
    # return nan if all empty list
    return np.nan if not valid_values else '|'.join(valid_values)

In [15]:
comb['gene'] = comb.apply(lambda r: join_columns(r, ['gene_psp','gene_ochoa']), axis=1)

comb['source'] = comb.apply(lambda r: join_columns(r, ['source_psp','source_ochoa']), axis=1)

comb['site_seq'] = comb.apply(lambda r: join_columns(r, ['site_seq_psp','site_seq_ochoa']), axis=1)

In [16]:
comb = comb[['uniprot','gene','site','site_seq','source']]

In [17]:
comb.source.value_counts()

source
ochoa        106327
psp            9138
ochoa|psp      5954
Name: count, dtype: int64

In [18]:
comb.shape

(121419, 5)

In [19]:
comb['position'] =comb.site.str[1:].astype(int)

comb = comb.sort_values(by=['uniprot', 'position']).reset_index(drop=True)

comb = comb.iloc[:,:5]

To save and load:

In [20]:
# comb.to_parquet('combine_site_psp_ochoa.parquet')

# comb = pd.read_parquet('combine_site_psp_ochoa.parquet')

## Query uniprot sequence on uniprot; mapping sequence

Uncheck below to use the csv for uniprot id mapping

In [21]:
# comb.drop_duplicates(subset='uniprot').to_csv('uniprot.csv',index=False)

In [22]:
# unmapped = pd.Series(['AAC50053',
# 'P18433-2',
# 'AAA58698',
# 'NP_001184222',
# 'AAA60149'])

In [23]:
sequence = pd.read_excel('raw/idmapping_2024_06_17.xlsx')

# there are few duplicates uniprot (history uniprot)
sequence = sequence.drop_duplicates(subset='uniprot')

In [24]:
seq = sequence[['uniprot','sequence']].copy()

In [25]:
comb = comb.merge(seq,how='inner',on='uniprot')

In [26]:
comb.shape

(121272, 6)

In [27]:
121419-121272 # unmatched

147

## Validate position

In [28]:
comb['position'] = comb.site.str[1:].astype(int)

In [29]:
comb['acceptor'] = comb.site.str[0]

In [30]:
def validate_position(row):
    # Extract amino acid and position from the new columns
    amino_acid = row['acceptor']
    position = int(row['position'])
    
    try:
        # Check if the amino acid at the given position matches the specified amino acid
        if row['sequence'][position-1] == amino_acid:
            return 1
        else:
            return 0
    except IndexError:  # Handle the case when position-1 exceeds the length of sequence
        return 0

In [31]:
comb['is_valid'] = comb.apply(validate_position,axis=1)

In [32]:
comb.is_valid.value_counts()

is_valid
1    120104
0      1168
Name: count, dtype: int64

In [33]:
comb = comb[comb.is_valid==1]

In [34]:
comb.source.value_counts()

source
ochoa        105775
psp            8382
ochoa|psp      5947
Name: count, dtype: int64

In [35]:
comb.shape

(120104, 9)

## Phosphorylate sequence

In [36]:
modify=comb.groupby('uniprot').agg({'site':lambda r: r.unique()}).reset_index()

In [37]:
modify = modify.merge(seq)

In [38]:
def phosphorylate_seq(row):
    seq = list(row['sequence'])
    for pos in row['site']:
        # extract character and position
        position = int(pos[1:]) - 1  # Subtracting 1 because Python uses 0-based indexing

        # convert sequence
        seq[position] = seq[position].lower()
    return ''.join(seq)

In [39]:
modify['phospho_seq'] = modify.apply(phosphorylate_seq,axis=1)

In [40]:
seq2 = modify[['uniprot','phospho_seq']]

## Extract sequence

In [41]:
comb = comb.merge(seq2)

In [42]:
comb.shape

(120104, 10)

In [43]:
site_seq = extract_site_seq(comb,'phospho_seq','position')

100%|██████████| 120104/120104 [00:03<00:00, 31171.64it/s]


In [44]:
comb['site_seq'] = site_seq

In [45]:
comb.shape

(120104, 10)

Reorder

In [46]:
comb['position'] =comb.site.str[1:].astype(int)

comb = comb.sort_values(by=['uniprot', 'position']).reset_index(drop=True)

comb = comb.iloc[:,:5]

To save and load:

In [47]:
# comb.to_parquet('phosphorylated_combine_site.parquet')

# comb=pd.read_parquet('phosphorylated_combine_site.parquet')

In [48]:
comb

,uniprot,gene,site,site_seq,source
0,A0A024R4G9,C19orf48,S20,ITGSRLLsMVPGPAR,psp
1,A0A075B6Q4,NaN,S24,VDDEKGDsNDDYDSA,ochoa
2,A0A075B6Q4,NaN,S35,YDSAGLLsDEDCMSV,ochoa
3,A0A075B6Q4,NaN,S57,IADHLFWsEETKSRF,ochoa
4,A0A075B6Q4,NaN,S68,KSRFTEYsMTssVMR,ochoa
...,...,...,...,...,...
120099,V9GYY5,NaN,S127,EGGAGDRsEEEAsst,ochoa
120100,V9GYY5,NaN,S132,DRsEEEAsstEKPtK,ochoa
120101,V9GYY5,NaN,S133,RsEEEAsstEKPtKA,ochoa
120102,V9GYY5,NaN,T134,sEEEAsstEKPtKAL,ochoa


## Access the dataset through `Data`

In [49]:
Data.get_combine_site_psp_ochoa()

,uniprot,gene,site,site_seq,source,AM_pathogenicity,CDDM_upper,CDDM_max_score
0,A0A024R4G9,C19orf48,S20,ITGSRLLSMVPGPAR,psp,NaN,"PRKX,AKT1,PKG1,P90RSK,HIPK4,AKT3,HIPK1,PKACB,H...",2.407041
1,A0A075B6Q4,None,S24,VDDEKGDSNDDYDSA,ochoa,NaN,"CK2A2,CK2A1,GRK7,GRK5,CK1G1,CK1A,IKKA,CK1G2,CA...",2.295654
2,A0A075B6Q4,None,S35,YDSAGLLSDEDCMSV,ochoa,NaN,"CK2A2,CK2A1,IKKA,ATM,IKKB,CAMK1D,MARK2,GRK7,IK...",2.488683
3,A0A075B6Q4,None,S57,IADHLFWSEETKSRF,ochoa,NaN,"GRK7,CK2A1,CK2A2,PKN2,GRK1,GRK5,MARK1,MARK2,UL...",1.851894
4,A0A075B6Q4,None,S68,KSRFTEYSMTSSVMR,ochoa,NaN,"AKT1,P90RSK,AKT3,SGK1,AKT2,NDR2,RSK2,P70S6K,RS...",2.026384
...,...,...,...,...,...,...,...,...
121414,V9GYY5,None,S127,EGGAGDRSEEEASST,ochoa,NaN,"CK2A1,CK2A2,GRK7,GRK5,ALK2,GRK1,CK1E,PLK3,CK1A...",2.665606
121415,V9GYY5,None,S132,DRSEEEASSTEKPTK,ochoa,NaN,"CK2A2,CK2A1,GRK7,TGFBR1,GRK2,ALK2,PLK3,CLK3,BM...",2.445179
121416,V9GYY5,None,S133,RSEEEASSTEKPTKA,ochoa,NaN,"CK2A1,ATR,GRK1,CK1G1,PLK3,CLK3,GRK7,CK1G2,MARK...",2.090739
121417,V9GYY5,None,T134,SEEEASSTEKPTKAL,ochoa,NaN,"ASK1,PERK,EEF2K,MAP2K4,MEKK2,MST1,BMPR1B,OSR1,...",1.832532


In [50]:
comb2 = Data.get_combine_site_phosphorylated()

In [51]:
comb2[comb2.gene=='CTNNB1']

,uniprot,gene,site,site_seq,source,AM_pathogenicity,CDDM,PSPA,CDDM_max_score,PSPA_max_score
28253,P35222,CTNNB1,T3,_____MAtQADLMEL,ochoa,0.350216,"ATR,ATM,DNAPK,CAMKK1,CAMKK2,PBK,ASK1,OSR1,TNIK...","MARK1,MARK2,DNAPK,ATR,SMG1,HUNK,QSK,MARK3,MARK...",1.685871,2.839518
28254,P35222,CTNNB1,S23,PDRKAAVsHWQQQsy,psp,0.434195,"P90RSK,RSK4,MARK1,RSK2,AKT1,TSSK2,SGK1,P70S6K,...","SSTK,BRSK1,PRKD3,BRSK2,P70S6K,SNRK,MARK3,MAPKA...",1.795827,3.255439
28255,P35222,CTNNB1,S29,VsHWQQQsyLDsGIH,ochoa|psp,0.628389,"PAK4,CAMK1D,NIM1,LATS2,PAK5,TBK1,TSSK1,GRK7,NU...","GSK3A,GSK3B,LATS2,MAPKAPK2,CAMK2A,CAMK2B,LATS1...",1.693298,6.465129
28256,P35222,CTNNB1,Y30,sHWQQQsyLDsGIHs,ochoa|psp,0.780358,"ERBB4,FGFR4,TNK1,JAK3,CSK,KIT,EPHA5,EGFR,JAK2,...","BMPR2_TYR,PTK2,SYK,ERBB4,PDHK1_TYR,EPHA3,PDHK4...",1.596045,2.796830
28257,P35222,CTNNB1,S33,QQQsyLDsGIHsGAT,psp,0.978753,"GSK3B,IKKB,IKKA,GSK3A,TBK1,PAK4,GRK1,IKKE,P90R...","CK1G2,CK1A,CK1G3,GSK3A,GSK3B,GRK3,CK1A2,CK1D,J...",1.797053,8.479370
28258,P35222,CTNNB1,S37,yLDsGIHsGATtTAP,ochoa|psp,0.954689,"GSK3A,PAK6,PAK5,GSK3B,TBK1,PAK4,PRKX,IKKB,ULK3...","GSK3A,GSK3B,CK1A,CK1G2,GRK7,IKKA,GRK4,GRK5,IKK...",1.795597,8.083523
28259,P35222,CTNNB1,T41,GIHsGATtTAPsLsG,psp,0.903105,"MPSK1,GSK3B,PBK,GSK3A,MEK2,ASK1,LKB1,TNIK,MEKK...","GSK3A,GSK3B,PRP4,PASK,CK1G2,CK1A,GRK7,CK1D,CK1...",1.649193,7.866048
28260,P35222,CTNNB1,S45,GATtTAPsLsGKGNP,ochoa|psp,0.915674,"TBK1,IKKE,PAK6,MTOR,PAK5,PRKX,RSK4,RSK2,CK1A,ULK3","CK1A,CK1G1,CK1D,CK1G2,MTOR,CK1E,CK1A2,CLK3,IKK...",2.044766,4.275749
28261,P35222,CTNNB1,S47,TtTAPsLsGKGNPEE,ochoa,0.556963,"CK1G2,PAK5,PAK6,PRKX,PKACB,CK2A1,PRKD3,ULK3,RS...","MPSK1,CK1E,ERK5,GRK1,CK1D,COT,MOS,NLK,MLK3,ERK1",1.823853,2.982430
28262,P35222,CTNNB1,S60,EEEDVDTsQVLyEWE,psp,0.208932,"ATM,ATR,GRK7,GRK1,BUB1B,CK2A2,GRK5,DNAPK,CK2A1...","ATM,ACVR2B,ACVR2A,GRK4,GRK7,CK1G2,PLK1,TLK2,PL...",2.656530,5.809839


## (code only) Add AlphaMissense score to each site

The AM_mean.parquet file is too big to upload to the current repository. To generate the file, refer to `others_01_Process_AM.ipynb` notebook.

In [22]:
df = pd.read_parquet('raw/AM_mean.parquet')

In [23]:
df.columns = ['uniprot','site','AM_pathogenicity','position']

In [24]:
df = df.iloc[:,:3]

The original:

In [12]:
comb1 = Data.get_combine_site_psp_ochoa()

In [26]:
comb1=comb1.merge(df,how='left',on=['uniprot','site'])

The phosphorylated

In [13]:
comb2 = Data.get_combine_site_phosphorylated()

In [32]:
comb2=comb2.merge(df,how='left',on=['uniprot','site'])

## Add kinase prediction

for uppercase, only cddm

In [21]:
comb1['site_seq'] = comb1['site_seq'].str.split('|').str[0] # only one site with one aa difference

In [45]:
cddm_upper = predict_kinase_df(comb1,'site_seq',**param_CDDM_upper)

input dataframe has a length 121419
Preprocessing
Finish preprocessing
Calculating position: [-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7]


100%|██████████| 289/289 [02:22<00:00,  2.03it/s]


In [24]:
def get_top(r, n):
    top = r.sort_values(ascending=False)[:n].index
    return ','.join(top)

In [25]:
cddm_upper_rnk = cddm_upper.apply(lambda r: get_top(r,10),axis=1)

In [27]:
comb1['CDDM_upper']=cddm_upper_rnk

In [46]:
comb1['CDDM_max_score'] = cddm_upper.max(1)

Uncheck below to save:

In [1]:
# comb1.to_parquet('raw/combine_site_psp_ochoa.parquet')

For phosphorylated:

In [32]:
cddm = predict_kinase_df(comb2,'site_seq',**param_CDDM)

input dataframe has a length 120104
Preprocessing
Finish preprocessing
Calculating position: [-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7]


100%|██████████| 289/289 [02:17<00:00,  2.11it/s]


In [33]:
pspa = predict_kinase_df(comb2,'site_seq',**param_PSPA)

input dataframe has a length 120104
Preprocessing
Finish preprocessing
Calculating position: [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]


100%|██████████| 396/396 [04:16<00:00,  1.55it/s]


In [34]:
cddm_rnk = cddm.apply(lambda r: get_top(r,10),axis=1)
pspa_rnk = pspa.apply(lambda r: get_top(r,10),axis=1)

In [35]:
comb2['CDDM']=cddm_rnk

In [39]:
comb2['PSPA']=pspa_rnk

In [47]:
comb2['CDDM_max_score']=cddm.max(1)
comb2['PSPA_max_score']=pspa.max(1)

To save and load:

In [ ]:
# comb2.to_parquet('raw/phosphorylated_combine_site.parquet')

# comb2 = pd.read_parquet('raw/phosphorylated_combine_site.parquet')